<a href="https://colab.research.google.com/github/3698atul/student-performance-monitor-workflow/blob/main/Mental_Health_Support_BOT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai gradio
import os
import gradio as gr
import google.generativeai as genai
from IPython.display import Markdown


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 7.4 MB/s eta 0:00:00


In [8]:

    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    genai.configure(api_key=GEMINI_API_KEY)


In [9]:
model = genai.GenerativeModel('gemini-2.5-pro')


def generate_response(prompt_text):
    """
    Generates a response from the Gemini model based on a prompt.
    """
    try:
        response = model.generate_content(prompt_text)
        return response.text
    except Exception as e:
        return f"An error occurred: {e}"

def get_mood(user_input):
    """
    Agent: Mood Analyzer
    Analyzes the user's input to determine their mood.
    """
    prompt = f"""
    Analyze the sentiment and emotional state of the following text and return a single word that best describes the mood.
    Examples:
    Text: "I'm feeling so overwhelmed with everything."
    Mood: Overwhelmed

    Text: "Today was a really great day! I got so much done."
    Mood: Happy

    Text: "I don't know what to do anymore. I just feel so lost."
    Mood: Sad

    Text: "{user_input}"
    Mood:
    """
    return generate_response(prompt)

def get_self_care_suggestion(mood):
    """
    Agent: Self-Care Recommender
    Suggests a mental wellness activity based on the detected mood.
    """
    prompt = f"""
    Based on the following mood, suggest a simple and helpful mental wellness or self-care activity. Be brief and encouraging.
    Mood: {mood}
    Example:
    Mood: Overwhelmed
    Suggestion: Try a 5-minute guided breathing exercise to recenter yourself.

    Mood: Anxious
    Suggestion: Write down all your worries in a journal, then try to reframe one of them.

    Mood: Sad
    Suggestion: Listen to some calming music and allow yourself to feel your emotions.

    Mood: Happy
    Suggestion: Take a moment to appreciate what's going well.

    Suggestion:
    """
    return generate_response(prompt)

def get_companion_response(user_input, mood):
    """
    Agent: Companion
    Generates an empathetic and supportive conversational response.
    """
    prompt = f"""
    You are a supportive and empathetic mental wellness companion. Your goal is to provide a non-judgmental and caring response.
    Keep your responses concise and conversational. Do not give medical advice.
    The user's current mood is detected as: {mood}.
    User: {user_input}
    Response:
    """
    return generate_response(prompt)

#Main chatbot function for Gradio
def chatbot_response(history):
    """
    Processes the user's latest message and generates a full response.
    """
    user_message = history[-1][0]

    # Agent 1 Mood Analyzer
    mood = get_mood(user_message)

    # Agent 2 Companion
    companion_response = get_companion_response(user_message, mood)

    # Agent 3 Self-Care Recommender
    if mood.strip().lower() in ['overwhelmed', 'anxious', 'sad', 'stressed']:
        wellness_suggestion = get_self_care_suggestion(mood)
        full_response = f"{companion_response}\n\n**Wellness Tip:** {wellness_suggestion}"
    else:
        full_response = companion_response

    # Update the last message in history with the bot's response
    history[-1][1] = full_response
    return history

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# Mental Health Support Bot 🤖💚")
    gr.Markdown("Hi there! I'm here to listen. You can share what's on your mind, and I'll do my best to provide a supportive response and a helpful wellness tip.")

    chatbot = gr.Chatbot(label="Chatbot")
    msg = gr.Textbox(placeholder="How are you feeling today?", label="Your Message")
    clear = gr.ClearButton([msg, chatbot])

    # Function to update the chat history with the user's message
    def add_message(user_message, history):
        return "", history + [[user_message, None]]

    # Gradio event handling
    msg.submit(add_message, [msg, chatbot], [msg, chatbot], queue=False).then(
        chatbot_response, chatbot, chatbot
    )

demo.launch(debug=True)

/tmp/ipython-input-3428232004.py:104: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chatbot")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e8db2311f26e69df74.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e8db2311f26e69df74.gradio.live
